In [77]:
import numpy as np
import pandas as pd
import time
from datetime import datetime

In [140]:
prod_filepath = "./data/production_data.csv"
df_prod = pd.read_csv(prod_filepath,low_memory=False)

alias_filepath = "./data/operator_aliasing.csv"
df_alias = pd.read_csv(alias_filepath,low_memory=False)

wells_filepath = "./data/wellsData.csv"
df_wells = pd.read_csv(wells_filepath,low_memory=False)

### Adjusting API by adding a leading zero to APIs starting with '5'

In [141]:
start = time.time()

# REMOVING DUPLICATES
df_prod = df_prod.drop_duplicates()

# summing Liquid and gas production for wells with multiple records for the same month and year
df_prod = df_prod.groupby(['API','Month','Year']).agg({'Liquid':'sum', 'Gas':'sum','RatioGasOil':'sum','Water':'sum','PercentWater':'sum','DaysOn':'sum','_LastUpdate':'first'}).reset_index()

df_prod.API=df_prod.API.apply(lambda value: str(value).zfill(14))
df_wells.API=df_wells.API.apply(lambda value: str(value).zfill(14))

### Add Operator Alias

In [142]:
filter_col = np.append(df_prod.columns, ['Alias'])

wells_alias_df = df_alias[['Name', 'Alias']].merge(df_wells[['operatorNameIHS', 'API']], left_on='Name', right_on='operatorNameIHS', how='right')
new_df = (df_prod.merge(wells_alias_df[['operatorNameIHS', 'Alias', 'API']], on='API', how='left'))

new_df = new_df.fillna({'Alias':new_df.operatorNameIHS})[filter_col]



### Remove Pre-Peak Months

In [143]:
# filtering out post-peak records with 0 oil production
# tmp_df = new_df[filter_col][new_df['Liquid']!=0]

# getting peak records
new_df = new_df.sort_values(['Year','Month'])
peak_records = new_df[new_df.groupby(['API'], sort=False)['Liquid'].transform(max) == new_df['Liquid']]

# filtering out pre-peak months
peak_records = peak_records.rename(columns={"Month": "PeakMonth", "Year": "PeakYear"})
new_df_with_peak = pd.merge(new_df, peak_records[["API", "PeakMonth", "PeakYear"]], on='API', how='left')
post_peak = new_df_with_peak[np.logical_or((new_df_with_peak['Year'] > new_df_with_peak['PeakYear']),np.logical_and((new_df_with_peak['Year'] == new_df_with_peak['PeakYear']),(new_df_with_peak['Month'] >= new_df_with_peak['PeakMonth'])))][filter_col]


### Add a Month Index Column

In [144]:
post_peak['MonthIdx'] = post_peak.sort_values(['Year','Month']).groupby(['API'], sort=False).cumcount()+1
end = time.time()
tt = end - start
print(tt)

45.05917501449585


In [145]:
post_peak.to_csv("./data/processed_production_data.csv")
# df_wells.to_csv("./data/processed_wells_data.csv")